# Evolution of agents - MCP

MCP (Model Context Protocol) is an open spec that standardizes how AI clients (agents, IDEs, chat apps) discover and call external tools and access resources. It uses a client–server model where servers expose capabilities (tools/functions, prompts, resources like files/DBs) over JSON-RPC transports (e.g., stdio, WebSocket). The goal is portability and safety: tools are reusable across different LLM runtimes with consistent schemas, auth, and streaming I/O, while remaining sandboxed. In practice, you run an MCP server for a service and any MCP-compatible client can list its capabilities and invoke them.

<img src="./images/agent_actions.png" alt="Agent with Actions" style="max-height: 300px;" />


In [ ]:
# Example: Inference using Semantic Kernel
from semantic_kernel import Kernel
from setup import get_project_client, create_agent, test_agent
from semantic_kernel.connectors.mcp import MCPStdioPlugin

client = await get_project_client()

kernel = Kernel()

async with MCPStdioPlugin(
    name="Playwright",
    command="npx",
    args=["@playwright/mcp@latest"],
    description="MCP Stdio Plugin for Playwright",
    version="1.0.0",
) as playwright_mpc:
    kernel.add_plugin(playwright_mpc, plugin_name="playwright")

    agent = await create_agent(
        agent_name="CalculatorAgentWithMcp",
        agent_instructions="You are a helpful assistant. Use tools to solve user queries.",
        client=client,
        kernel=kernel,
        plugins=[playwright_mpc],
    )

    thread = None
    user_input = "go to https://reindeerromp5k.com/ and check dates, prices and details for the race"
    thread = await test_agent(client, agent, user_input, thread)